# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

### Streaming tick data

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2018, 6, 6, 13, 54, 44, 80796, tzinfo=datetime.timezone.utc), bid=1.17911, bidSize=15000000, ask=1.17914, askSize=1000000, prevBid=1.17913, prevBidSize=15500000, prevAsk=1.17915, prevAskSize=3500000, high=1.17955, low=1.1714, close=1.1718, ticks=[TickData(time=datetime.datetime(2018, 6, 6, 13, 54, 44, 80796, tzinfo=datetime.timezone.utc), tickType=2, price=1.17914, size=1000000), TickData(time=datetime.datetime(2018, 6, 6, 13, 54, 44, 80796, tzinfo=datetime.timezone.utc), tickType=3, price=1.17914, size=1000000)], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.179125

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,6500000,1.17906,1.17908,1000000,1.17955,1.1714,1.1718
1,USDJPY,7000000,109.959,109.96,1000000,110.23,109.79,109.8
2,GBPUSD,500000,1.34383,1.34387,1000000,1.34435,1.33965,1.3397
3,USDCHF,2000000,0.98441,0.98444,1000000,0.9886,0.98355,0.98465
4,USDCAD,6000000,1.2872,1.28724,2000000,1.2974,1.2857,1.2968
5,AUDUSD,2000000,0.76747,0.7675,4500000,0.76765,0.7615,0.76165


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [8]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2018, 6, 6, 13, 54, 34, tzinfo=datetime.timezone.utc), mask=0, priceBid=1.17906, priceAsk=1.17908, sizeBid=3500000, sizeAsk=2000000)

In [9]:
ib.disconnect()